In [ ]:
# Raw IMU & Audio CSV → Pickle Audio Preprocessing
import os
import pickle
from pathlib import Path

import numpy as np
import pandas as pd
from utils import vggish_input

# Configuration
DATA_ROOT = Path("../../Data/Pilot_Data/1. RawDataset")
SAVE_ROOT = Path("../../Data/Pilot_Data/3. PreprocessDataset_for_Audio")
PARTICIPANT = '16'
IMU_SUFFIX = 'SensorData.csv'
AUDIO_SUFFIX = 'AudioData.csv'
ANNO_SUFFIX = 'annotation_modified.csv'
SUB_SR = 16000
LEH, UEH = 10, SUB_SR//2

# Utilities
def add_time_zero(df, time_col='Time', ux_col='UnixTime'):
    if (df[time_col]==0).any():
        return df
    first = df.iloc[0].copy()
    new = first.copy()
    new[time_col]=0
    new[ux_col]=first[ux_col] - int(first[time_col]*1000)
    return pd.concat([new.to_frame().T, df], ignore_index=True)


def load_data():
    base = DATA_ROOT / PARTICIPANT
    imu_file = next(p for p in base.iterdir() if p.name.endswith(IMU_SUFFIX))
    audio_file = next(p for p in base.iterdir() if p.name.endswith(AUDIO_SUFFIX))
    anno_file = next(p for p in base.iterdir() if p.name.endswith(ANNO_SUFFIX))

    df_imu = pd.read_csv(imu_file); df_imu = add_time_zero(df_imu)
    df_audio = pd.read_csv(audio_file); df_audio = add_time_zero(df_audio)
    df_anno = pd.read_csv(anno_file)

    # align annotations to IMU time zero
    imu0 = df_imu.loc[df_imu.Time==0,'UnixTime'].iloc[0]
    anno0 = df_anno.loc[df_anno.Time==0,'UnixTime'].iloc[0]
    df_anno.UnixTime += (imu0 - anno0)
    # end time
    end = df_anno.loc[df_anno.Event=='session_stop','UnixTime'].iloc[0]

    # trim IMU & audio to session
    df_imu = df_imu[df_imu.UnixTime<=end]
    df_audio = df_audio[df_audio.UnixTime<=end]
    return df_imu, df_audio, df_anno


def extract_intervals(df_anno):
    df = df_anno[df_anno.Event.isin(['start','stop'])]
    stack={} ; intervals=[]
    for _,r in df.iterrows():
        if r.Event=='start': stack[r.Activity]=r.UnixTime
        elif r.Event=='stop' and r.Activity in stack:
            intervals.append((stack.pop(r.Activity), r.UnixTime, r.Activity))
    return intervals


def label_audio(df_audio, intervals):
    df = df_audio.copy()
    df['Activity']='Other'
    audio_cols = [c for c in df.columns if c.startswith('AudioData')]
    for st,et,act in intervals:
        mask = (df.UnixTime>=st)&(df.UnixTime<=et)
        df.loc[mask,'Activity']=act
    df['run_id'] = (df.Activity!=df.Activity.shift()).cumsum()
    samples, labels = [], []
    for _,grp in df.groupby('run_id'):
        arr = grp[audio_cols].to_numpy(dtype=np.int16).flatten()
        samples.append(arr); labels.append(grp.Activity.iloc[0])
    return samples, labels


def window_and_save(samples, labels):
    audio_windows=[]; audio_labels=[]
    for arr,label in zip(samples, labels):
        mel = vggish_input.wavform_to_examples(arr, lower_edge_hertz=LEH, upper_edge_hertz=UEH, sr=SUB_SR)
        audio_windows.append(mel)
        audio_labels += [label]*mel.shape[0]
    X = np.concatenate(audio_windows,axis=0)
    y = np.array(audio_labels)
    # save
    out_dir = SAVE_ROOT / PARTICIPANT
    out_dir.mkdir(exist_ok=True, parents=True)
    with open(out_dir/f'{PARTICIPANT}_preprocessing_for_audio.pkl','wb') as f:
        pickle.dump({'Audio':X,'Activity':y}, f)
    print(f"Saved {X.shape} examples with labels {set(y)}")


if __name__=='__main__':
    df_imu, df_audio, df_anno = load_data()
    intervals = extract_intervals(df_anno)
    samples, labels = label_audio(df_audio, intervals)
    window_and_save(samples, labels)
